In [1]:
# from lensing_pointmass import *
import numpy as np
import matplotlib.pyplot as plt
import bilby
import scipy.constants as constants
from scipy.integrate import simps
from scipy.interpolate import interp1d
from lal import MTSUN_SI, PC_SI
from astropy.cosmology import FlatLambdaCDM
from chainconsumer import ChainConsumer, Chain, ChainConfig, PlotConfig
import matplotlib
plt.rcParams.update({'font.size': 15})
matplotlib.rcParams['text.usetex'] = True

# %config InlineBackend.figure_format = 'retina'
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

/tmp/ipykernel_280563/2992806955.py:8: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import MTSUN_SI, PC_SI


In [2]:
#Geometric optics - multiple images
def t_delay_geom_plus(y):
    return -y-0.5
def t_delay_geom_minus(y):
    return y-0.5
def DeltaT(y):
    return t_delay_geom_minus(y)-t_delay_geom_plus(y)

def mu_plus(y):
    return 1+1/y
def mu_minus(y):
    return -1+1/y

def F_geom_opt(ws,y):
    Fplus = np.sqrt(np.abs(mu_plus(y)))*np.exp(1.0j*ws*t_delay_geom_plus(y))
    Fminus = np.sqrt(np.abs(mu_minus(y)))*np.exp(1.0j*ws*t_delay_geom_minus(y))*np.exp(-1.0j*(np.pi/2.)*np.sign(ws))
    return Fplus + Fminus

def theta_plus(y,thetaE):
    return (y+1)*thetaE
def theta_minus(y,thetaE):
    return (y-1)*thetaE

In [3]:
ligo_psd = np.loadtxt('../data/AplusDesign.txt')
f_interp=ligo_psd[:,0]
ligo_interp = interp1d(f_interp,ligo_psd[:,1]**2)

virgo_psd = np.loadtxt('../data/V1_O5_strain.txt')
virgo_interp = interp1d(virgo_psd[:,0],virgo_psd[:,1]**2)

kagra_psd = np.loadtxt('../data/K1_O5_strain.txt')
kagra_interp = interp1d(kagra_psd[:,0],kagra_psd[:,1]**2)

In [32]:
yh = 0.5
zL = 1.
zS = 2
H0 = 67.7
Om0 = 0.308
inc = np.pi/3
ML = 5e10

m1 = 36*(1+zS)
m2 = 29*(1+zS)
Mc = (m1*m2)**(3/5)/(m1 + m2)**(1/5)
q = m2/m1

f_low = 10
delta_f = 1./16
tc = 0
approx = 'IMRPhenomXPHM'

In [33]:
cosmo = FlatLambdaCDM(H0=H0,Om0=Om0)
DL = cosmo.angular_diameter_distance(zL).value
DS = cosmo.angular_diameter_distance(zS).value
DLS = cosmo.angular_diameter_distance_z1z2(zL,zS).value
print(DL,DS,DLS)

1700.0031617687716 1773.1807652282178 639.8453240490368


In [34]:
thetaE = np.sqrt(4*np.pi*ML*MTSUN_SI*constants.c/PC_SI *DLS/DS/DL/1e6)
print(np.rad2deg(thetaE))
print(np.rad2deg(thetaE)*3600) #arcsec

0.00014474665556689465
0.5210879600408207


In [35]:
theta_p = theta_plus(yh,thetaE)
theta_m = theta_minus(yh,thetaE)
print(np.rad2deg(theta_p)*3600,np.rad2deg(theta_m)*3600)

0.7816319400612312 -0.26054398002041035


In [36]:
np.rad2deg(theta_p-theta_m)*3600/2

0.5210879600408207

In [17]:
tplus = t_delay_geom_plus(yh)
tminus = t_delay_geom_minus(yh)
delt = DeltaT(yh)

print(4*np.pi*ML*MTSUN_SI*(1+zL)*tplus/3600, 4*np.pi*ML*MTSUN_SI*(1+zL)*tminus/3600, 4*np.pi*ML*MTSUN_SI*(1+zL)*delt/3600) #hr

-1891.2527548973464 0.0 1891.2527548973464


In [18]:
print(4*np.pi*ML*MTSUN_SI*(1+zL)*delt/3600/24) #days

78.80219812072276


In [21]:
print((1+zL)/constants.c*DL*DS/DLS*1e6*PC_SI*thetaE**2*delt/3600/24) #days

78.80219812072276


In [19]:
dls = cosmo.luminosity_distance(zS).value
print(dls)

1012.169692168754


In [23]:
duration = 1/delta_f
sampling_frequency = 4096
t_gps = 20000

parameters = dict(mass_1=m1, mass_2=m2, a_1=0, a_2=0, tilt_1=0, tilt_2=0, luminosity_distance=dls, theta_jn=inc, geocent_time=t_gps, phase=0)

waveform_arguments = dict(waveform_approximant=approx,minimum_frequency=f_low)

waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration, sampling_frequency=sampling_frequency,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=waveform_arguments)

freq0 = waveform_generator.frequency_array

h = waveform_generator.frequency_domain_strain(parameters=parameters)


18:21 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


In [24]:
def scalar_product(hf, gf, psd, freqs):
    return 2.*simps( np.real((hf*np.conjugate(gf)+np.conjugate(hf)*gf))/psd, x=freqs)

def Fisher_build(farr, psd, derivatives):

    Fisher_matrix = np.zeros((len(derivatives),len(derivatives)))

    for i in range(0, len(derivatives)):
        for j in range(0,i +1):
            Fisher_matrix[i,j] = scalar_product(derivatives[i], derivatives[j], psd, farr)
            Fisher_matrix[j,i] = Fisher_matrix[i,j]

    connum = np.linalg.cond(Fisher_matrix)
    print('condition number (div by 1e15)',connum/1.e15)

    return Fisher_matrix

In [25]:
ind = np.where(freq0>f_low)[0]
print(freq0[ind][0])

10.0625


In [53]:
ifos = bilby.gw.detector.InterferometerList(['L1','H1','V1','K1'])

ra = np.deg2rad(26)
dec = np.deg2rad(48)
print(ra,dec)
psi = 0
t_gps = 30000


0.4537856055185257 0.8377580409572782


In [57]:
h_det = []

for ifo in ifos:
    Fp = ifo.antenna_response(ra, dec, t_gps, psi, 'plus')
    Fx = ifo.antenna_response(ra, dec, t_gps, psi, 'cross')

    h_0 = Fp*h['plus']+Fx*h['cross']
    h_det.append(h_0)

snr_l1_sq = scalar_product(h_det[0][ind], h_det[0][ind], ligo_interp(freq0[ind]), freq0[ind])
snr_h1_sq = scalar_product(h_det[1][ind], h_det[1][ind], ligo_interp(freq0[ind]), freq0[ind])
snr_v1_sq = scalar_product(h_det[2][ind], h_det[2][ind], virgo_interp(freq0[ind]), freq0[ind])
snr_k1_sq = scalar_product(h_det[3][ind], h_det[3][ind], kagra_interp(freq0[ind]), freq0[ind])

snr = np.sqrt(snr_l1_sq+snr_h1_sq+snr_v1_sq+snr_k1_sq)
print(snr, np.sqrt(snr_l1_sq), np.sqrt(snr_h1_sq), np.sqrt(snr_v1_sq), np.sqrt(snr_k1_sq))

41.84933934284356 16.681422227826324 29.51493692622936 17.556551802451157 17.13865055767248


/tmp/ipykernel_909454/2320048601.py:2: DeprecationWarning: 'scipy.integrate.simps' is deprecated in favour of 'scipy.integrate.simpson' and will be removed in SciPy 1.14.0
  return 2.*simps( np.real((hf*np.conjugate(gf)+np.conjugate(hf)*gf))/psd, x=freqs)


In [66]:
t_delay = 4*np.pi*ML*MTSUN_SI*(1+zL)*delt

h_det = []

for ifo in ifos:
    Fp = ifo.antenna_response(ra, dec, t_gps+t_delay, psi, 'plus')
    Fx = ifo.antenna_response(ra, dec, t_gps+t_delay, psi, 'cross')

    h_0 = Fp*h['plus']+Fx*h['cross']
    h_det.append(h_0)

snr_l1_sq = scalar_product(h_det[0][ind], h_det[0][ind], ligo_interp(freq0[ind]), freq0[ind])
snr_h1_sq = scalar_product(h_det[1][ind], h_det[1][ind], ligo_interp(freq0[ind]), freq0[ind])
snr_v1_sq = scalar_product(h_det[2][ind], h_det[2][ind], virgo_interp(freq0[ind]), freq0[ind])
snr_k1_sq = scalar_product(h_det[3][ind], h_det[3][ind], kagra_interp(freq0[ind]), freq0[ind])

snr_d = np.sqrt(snr_l1_sq+snr_h1_sq+snr_v1_sq+snr_k1_sq)
print(snr_d, np.sqrt(snr_l1_sq), np.sqrt(snr_h1_sq), np.sqrt(snr_v1_sq), np.sqrt(snr_k1_sq))

39.71592260557938 16.591511038288004 27.15681109013631 16.65825241397161 16.943627350889102


/tmp/ipykernel_909454/2320048601.py:2: DeprecationWarning: 'scipy.integrate.simps' is deprecated in favour of 'scipy.integrate.simpson' and will be removed in SciPy 1.14.0
  return 2.*simps( np.real((hf*np.conjugate(gf)+np.conjugate(hf)*gf))/psd, x=freqs)


In [51]:
(t_delay/86400-int(t_delay/86400))*86400

69309.91763044681

In [56]:
np.sqrt(np.abs(mu_plus(yh))), np.sqrt(np.abs(mu_minus(yh)))

(1.7320508075688772, 1.0)